## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-17-23-52-23 +0000,nypost,"Nick Reiner does ‘drugs,’ threatens to kill do...",https://nypost.com/2025/12/17/us-news/nick-rei...
1,2025-12-17-23-51-02 +0000,nypost,Vilification of Jews has emboldened NYC bigots...,https://nypost.com/2025/12/17/us-news/vilifica...
2,2025-12-17-23-50-47 +0000,nypost,AOC fuels 2028 presidential buzz with one-word...,https://nypost.com/2025/12/17/us-news/aoc-fuel...
3,2025-12-17-23-49-54 +0000,nyt,House Passes Bill to Ban Gender Transition Tre...,https://www.nytimes.com/2025/12/17/us/politics...
4,2025-12-17-23-49-23 +0000,nyt,Republicans Clinch Democrats’ Bid to Force Vot...,https://www.nytimes.com/2025/12/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
185,trump,54
1,reiner,25
452,bondi,22
187,venezuela,20
189,oil,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
383,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...,148
81,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,141
134,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,134
404,2025-12-17-00-10-52 +0000,cbc,Trump orders blockade of 'sanctioned oil tanke...,https://www.cbc.ca/news/world/trump-venezuela-...,125
367,2025-12-17-02-58-00 +0000,wsj,Trump Orders Blockade of Sanctioned Oil Tanker...,https://www.wsj.com/world/americas/trump-order...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
383,148,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...
189,103,2025-12-17-17-47-56 +0000,bbc,"What we know about Nick Reiner, son charged wi...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
134,80,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
138,76,2025-12-17-19-55-55 +0000,nypost,Top cop defends Bondi Beach officer accused of...,https://nypost.com/2025/12/17/world-news/top-c...
263,72,2025-12-17-12-59-17 +0000,nypost,Warner Bros. Discovery board urges shareholder...,https://nypost.com/2025/12/17/business/warner-...
361,61,2025-12-17-03-40-00 +0000,wsj,The Senate majority leader said Congress won’t...,https://www.wsj.com/politics/policy/house-gop-...
120,52,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
380,46,2025-12-17-02-00-00 +0000,wsj,Tuesday’s jobs report was the latest ominous s...,https://www.wsj.com/economy/jobs/white-collar-...
6,42,2025-12-17-23-48-04 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/12/17/us-news/tyler-ro...
50,38,2025-12-17-22-31-50 +0000,nypost,Providence police admit Brown University mass ...,https://nypost.com/2025/12/17/us-news/providen...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
